In [1]:
from FASTSim_Package.envs.FASTSim_RL_Environment import *
from utils import *   # import replay buffer for experienced replay and plotting function
from DQN_Agent import *
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style

style.use('ggplot')

In [2]:
env = gym.make('FASTSim-v0')
a = env.action_space
print(env.state_space, a)

Box(2,) Discrete(11)


In [3]:
if __name__ == '__main__':
    env = gym.make('FASTSim-v0')
    n_games = 151
    agent = Agent(gamma=0.99, lr=0.0005, input_dims=2, n_actions=11, batch_size=64, 
                  epsilon=1.0, eps_dec=0.9999995, eps_min=0.0, mem_size=1000000)
        
    scores = []
    eps_history = []

    for i in range(n_games):
        done = False
        score = 0
        state = env.reset()
        while not done:
            action = agent.choose_action(state)
            state_, reward, done, info = env.step(action)
            score += reward
            agent.remember(state, action, reward, state_, done) # store experience
            state = state_   # new state -- state
            agent.learn()   
            
        eps_history.append(agent.epsilon)
        scores.append(score)

        avg_score = np.mean(scores[max(0, i-100):(i+1)])
        print('episode: ', i,'score: %.2f' % score,
              ' average score %.2f' % avg_score)

        if i % 10 == 0 and i > 0:
            agent.save_model()

    filename = 'FASTSim.png'

    x = [i+1 for i in range(n_games)]
    plotLearning(x, scores, eps_history, filename)

action_indices: [ 3  4  0  9  9  8  6  9  6  7  8  8 10  1  1  7  3  7  9  9  1  4  8  1
  8  7  8 10  9 10  7  7  3  2  5  5 10  7  3  5 10  6  8  0  4  3  4  6
  2  8  2  6  0  3  2  9  4  4  2  1  3  4  9  5] action_values: [ 0  1  2  3  4  5  6  7  8  9 10]
action_indices: [ 3  7  3  1  3  3  2 10  2  6  8  4  9  1  5  4  4  8  1  6  9  7  4  6
  7  1  8  7  8  2  1  0  9  6  7  3  9  9  9  6  8 10  0  4  4  8  4  3
  8  9  8  7  6  7 10 10  5  2  5  7  5 10  3  0] action_values: [ 0  1  2  3  4  5  6  7  8  9 10]
action_indices: [ 8  7  4  3  8  4  4  9  4  2  9  3  3  6  6  2  8  6  5  9  3  9  5  5
  7  4  2  1 10  9  7  6  3  9  2 10 10  2 10  0  9  9  1  7  6  8  9  8
  0  6  8  4  7  0  1 10  3  1  8  4  1  7  8  5] action_values: [ 0  1  2  3  4  5  6  7  8  9 10]
action_indices: [ 3  4 10  8  6  6  5  4  4  7  5  7  9  6  8  9 10  0  5  3  7 10  3  9
  8  2  3 10  1  5  1  3  1  4  7  4  5  3  0  2  2  8  8  9  7  9  7  9
  9  4  6  4  8  9  8  0  7  2 10  8  6  7  3  9] ac

action_indices: [ 8  6  2  5  3  3  7 10  6  6  4  5  0 10  5  0  1  5  3  9  9  9  6  3
  8  1  3  4  4  8  7  1  3  8  2  4  7  1  8  9  0  5  2  5  1  8  7  3
  8 10  6 10  0  3  5  7  9 10  9  7  9 10  7  8] action_values: [ 0  1  2  3  4  5  6  7  8  9 10]
action_indices: [ 7 10  5  1  9  9  7  6  9  5  5  4  8  0  5  0  0  5  9  6  8  4  3  5
  7  4  7  8  8  6  8  9  9  4  2  9 10  6  8  7  3  8  7  8  2  1  2  3
  7  9  1  5  9  3  4  7  9  7  1  0  3  2  1  4] action_values: [ 0  1  2  3  4  5  6  7  8  9 10]
action_indices: [ 7  5 10  7  7  8  8  3  8  7  3  2  7 10  7  9  9  8  1  9 10  5  4  9
 10  6  8  9  0  5  2  9  7  4  0  6  1  5  9  3  5  6  4  8  5  6  1  7
  1  6  9  2  7  4  1  2  2  7  0  3  4  3  8  0] action_values: [ 0  1  2  3  4  5  6  7  8  9 10]
action_indices: [ 7  5  4  4  2  5  9  6  5  7  8  3  5  1  9  5  3  9  8  7  7  8  5  5
  7  8  3  0  4  2  9  3  1  9  4  6  4  3  7  6  5  9  1  8  3  6  8  6
  8 10  4  1  9 10 10  0  2  4  8  4  7  8 10  9] ac

action_indices: [ 5  3  3  8  1  9  3  8  3  7  9  2  1  4 10  4  4  6  3 10  7 10  1  1
  5  4  3 10  7  6 10  7  2  6  7  4  9  2  9  6  7  6  3  8  5  9  8  9
  2  4  8  5  6  7 10  9  0  9 10  7  9  8  1  5] action_values: [ 0  1  2  3  4  5  6  7  8  9 10]
action_indices: [ 3  8  7  5  2  0  4  4  9  6 10  9  7  5 10  3 10 10  4  5  6  6  1  3
  3  8  5  8  9  8  3  9  9  7  8  6  5  4  0  0  9  9  6  1  5  8  1  6
  8  4  1  4  5  9  8  6  6  3  6  3  6  3  6  8] action_values: [ 0  1  2  3  4  5  6  7  8  9 10]
action_indices: [ 2  4  6  7  9  3  1  9  8 10  5  8 10  7  2  1 10  5  9  9  9  4  1  5
  5  6  2  1 10  3  6  1  3  3  8  4  3  0  3  6  6  5  9  4  7  7  3  5
 10 10  8  0  4  0  1  7  6  6  1  2  8  5  9  8] action_values: [ 0  1  2  3  4  5  6  7  8  9 10]
action_indices: [ 9  2  3  2  9  9  7  4  8  3  5  7  6  0  6  5  8 10  3 10  9  3  0 10
  5  8  7  7  4  4  7  9  4  5  8  6  6  0  7 10  3  6  1  0  5  6  1  7
  9  8  5  2  1 10  9  5  1  9  9  0  6  9  3  1] ac

action_indices: [ 5  4  1  9  6  5  1  0 10  4  8  3 10  7  2  9  5  7  8  8  2  2  9  2
  4  4  1  3  6  2  2  9  3  3  3  7  5  9  8  1  6  0  7 10 10  4  6  8
  1  0  8  0  7  5 10  9  0  7  4  6 10  3  7  2] action_values: [ 0  1  2  3  4  5  6  7  8  9 10]
action_indices: [ 5  6  5  1  7  6  6  1  1  4  5  1  8  4  4  2  4  9  4  1  2  1  2  8
  3  5  2  9  7  1  6  6  9  3  3  5  8 10  8  1  0  5  6  8  4 10  4  7
 10  9  7  6  4  8  0  7  4  7  9  9  7 10  9  3] action_values: [ 0  1  2  3  4  5  6  7  8  9 10]
action_indices: [ 8  4  5  1  8  6  0  7  4  1  2  9  1  1  0  8  9  7  6  4  4  4  2  3
  3  1  3 10  6 10  4  6  7  7 10  7  7  2  9 10  6  1  9 10  4  6  6  6
 10  4  4  7  2  0 10  7  6  5  8  2  3  1  5  3] action_values: [ 0  1  2  3  4  5  6  7  8  9 10]
action_indices: [ 0  6  3  1  3  1  5  4  3  1  9  3  4  3  8  5  8  8  7  5  4  4 10  9
  5  9  7  0  5  5 10  1  6  7  7  4  3  8  2  1  2  5  1  5  3  8  2  8
  9  2  9  4  9  3  6  5  5  9  3  9  8  8  1  4] ac

KeyboardInterrupt: 